# 

# Predictions

## Post-training analysis

In [ ]:
import matplotlib as plt
import numpy as np
import pandas as pd
import seaborn as sns

dataset_train = f'../datasets/dataset_train.csv'
df = pd.read_csv(dataset_train)
df.drop(df.columns[2:6], inplace=True, axis = 1)
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df.drop(excluded_features, inplace=True, axis=1)
df.dropna(inplace=True)
df.head()

In [ ]:
df_x_train = 
df_x_train

In [ ]:
    print(self.x_train.shape, self.df_weights.shape)
    z = np.dot(self.x_train, self.df_weights)
    h = logreg.sigmoid(z)
    df_pred_proba = pd.DataFrame(h, columns=self.houses)
    df_pred_proba['Predicted outcome'] = df_pred_proba.idxmax(axis=1)
    df_pred_proba['Real outcome'] = self.df_class.tolist()
    accurate_pred = np.where(df_pred_proba['Predicted outcome']
                                == df_pred_proba['Real outcome'], 1, 0)
    df_pred_proba['Accurate pred.'] = accurate_pred